In [10]:
import re
text = "Hello, world. This is a Test-- ?."
result = re.split(r'([,.:;?_!()\']|--|\s)',text)

In [11]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', 'is', 'a', 'Test', '--', '?', '.']


In [12]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [13]:
## Tokenization

preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [14]:
print(len(preprocessed))

4690


# Creating token ids

In [24]:
## create a list of all tokens and sort them to determine the vocabulary size
tokens = sorted(set(preprocessed))
all_tokens = tokens.extend(['<|endoftext|>','<|unk|>'])
vocab_size = len(tokens)
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 1132


In [25]:
## now creating vocab i.e. a dictionary that maps each token to a unique integer
vocab = {token:integer for integer,token in enumerate(tokens)}

## print the first 10 tokens and their corresponding ids
for token, integer in list(vocab.items())[:10]:
    print(f"{token}: {integer}")

!: 0
": 1
': 2
(: 3
): 4
,: 5
--: 6
.: 7
:: 8
;: 9


In [26]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {integer:token for token,integer in vocab.items()}
    
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        return [self.str_to_int[token] for token in preprocessed]
    
    def decode(self,tokens):
        text = " ".join([self.int_to_str[token] for token in tokens])
        ## replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text
    

In [27]:
tokenizer = SimpleTokenizerV1(vocab)

## encode a sample text
text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

## decode the ids back to text
decoded_text = tokenizer.decode(ids)
print(decoded_text)


[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [30]:
## fails to tokenize word not in vocab
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {integer:token for token,integer in vocab.items()}
    
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [token if token in self.str_to_int else '<|unk|>' for token in preprocessed]
        return [self.str_to_int[token] for token in preprocessed]
    
    def decode(self,tokens):
        text = " ".join([self.int_to_str[token] for token in tokens])
        ## replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?_!"()\']|--)', r'\1', text)
        return text
    

In [31]:
tokenizer = SimpleTokenizerV2(vocab)

## encode a sample text
text = """"Hello, world. This is a Test-- ?."""

ids = tokenizer.encode(text)
print(ids)

## decode the ids back to text
decoded_text = tokenizer.decode(ids)
print(decoded_text)


[1, 1131, 5, 1131, 7, 97, 584, 115, 1131, 6, 10, 7]
" <|unk|>, <|unk|>. This is a <|unk|>--?.


In [36]:
text1 = "Hello, world. This is a Test-- ?."
text2 = "In the sunlit meadows, the flowers bloom."

## when we have 2 text source, we join them with a special token
text = text1 + " <|endoftext|> " + text2

print(text)

ids = tokenizer.encode(text)
print(ids)

Hello, world. This is a Test-- ?. <|endoftext|> In the sunlit meadows, the flowers bloom.
[1131, 5, 1131, 7, 97, 584, 115, 1131, 6, 10, 7, 1130, 55, 988, 956, 1131, 5, 988, 449, 1131, 7]


In [37]:
decoded_text = tokenizer.decode(ids)
print(decoded_text)

<|unk|>, <|unk|>. This is a <|unk|>--?. <|endoftext|> In the sunlit <|unk|>, the flowers <|unk|>.
